In [1]:
import sys
import json
import pandas as pd
import numpy as np

In [85]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import string

In [2]:
review_fn = "/Users/evan/msds_main/machine_learning_1/yelp/yelp_dataset/yelp_academic_dataset_review.json"

with open(review_fn, 'r', encoding='UTF-8') as f:
    rev_data = f.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    rev_data = list(map(json.loads, rev_data))
    
rev_df = pd.DataFrame(rev_data)

business_fn = '/Users/evan/msds_main/machine_learning_1/yelp/yelp_dataset/yelp_academic_dataset_business.json'

with open(business_fn, 'r', encoding='UTF-8') as f:
    bus_data = f.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    bus_data = list(map(json.loads, bus_data))

In [3]:
bus_df = pd.DataFrame(bus_data)[['business_id','categories','attributes','city','name','review_count','stars']]

In [4]:
rev_df.rename(columns = {'stars':'review_stars'}, inplace = True)
bus_df.rename(columns = {'stars':'rest_stars'}, inplace = True)

,business_id,cool,date,funny,review_id,review_stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g


In [6]:
bus_df = bus_df[bus_df.categories.str.contains('Restaurant|Food', na=False)]
bus_df = bus_df[bus_df.review_count > 25]

In [7]:
# Create happy hour label from attributes data
# bus_df['happy_hour'] = [True if attrs is not None and "HappyHour" in attrs and attrs["HappyHour"] == "True" else False for attrs in bus_df['attributes']]

In [8]:
rest_df = rev_df.merge(bus_df, how='inner', left_on='business_id', right_on='business_id')

,business_id,cool,date,funny,review_id,text,useful,user_id,categories,attributes,city,name,review_count,rest_stars,happy_hour
review_stars,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,465405,465405,465405,465405,465405,465405,465405,465405,465405,463208,465405,465405,465405,465405,465405
2,373063,373063,373063,373063,373063,373063,373063,373063,373063,372072,373063,373063,373063,373063,373063
3,539889,539889,539889,539889,539889,539889,539889,539889,539889,538601,539889,539889,539889,539889,539889
4,1064126,1064126,1064126,1064126,1064126,1064126,1064126,1064126,1064126,1061977,1064126,1064126,1064126,1064126,1064126
5,1576241,1576241,1576241,1576241,1576241,1576241,1576241,1576241,1576241,1572090,1576241,1576241,1576241,1576241,1576241


In [42]:
rest_df['user_review_count'] = df.groupby(['user_id'])['review_id'].transform('count')

,business_id,cool,date,funny,review_id,review_stars,text,useful,user_id,categories,attributes,city,name,review_count,rest_stars,happy_hour,stars_delta,avg_star_delta,user_review_count
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,9.0,4.0,False,-2.0,-0.5000,9.0
1,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-10-25,0,VluIpojME0yKOcRKI5L0Iw,3,"came here on Monday, no line. its good, but no...",0,D1_nrBr4dOrs7M82OaBRwQ,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,8.0,4.0,False,-1.0,0.5625,8.0
2,iCQpiavjjPzJ5_3gPD5Ebg,0,2016-05-22,0,nsB9JAeghk0zOaSulSm9Yw,5,This place is truly a secret! Its so hidden t...,0,jEjc8HxeCeI22M4di0Y1aw,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,1.0,4.0,False,1.0,1.0000,1.0
3,iCQpiavjjPzJ5_3gPD5Ebg,0,2014-12-30,0,pcn01EMERpCpHEcoaohdEg,5,This SECRET PIZZA was a secret for about 5 st...,0,RTiaaHSN80VRTLSdNK5rfQ,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,1.0,4.0,False,1.0,1.0000,1.0
4,iCQpiavjjPzJ5_3gPD5Ebg,0,2013-03-24,0,0mFtAPTmInbXHqDjX9eiOg,5,this is the BEST place to grab a quick slice o...,0,hBmJs49zYcXdV4wo8F5iiA,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,5.0,4.0,False,1.0,-0.1000,5.0


In [46]:
rest_df['stars_delta'] = rest_df.review_stars - rest_df.rest_stars
rest_df['avg_star_delta'] = rest_df.groupby(['user_id'])['stars_delta'].transform('mean')
rest_df.head()

,business_id,cool,date,funny,review_id,review_stars,text,useful,user_id,categories,attributes,city,name,review_count,rest_stars,happy_hour,stars_delta,avg_star_delta,user_review_count
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,9.0,4.0,False,-2.0,-0.5000,9.0
1,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-10-25,0,VluIpojME0yKOcRKI5L0Iw,3,"came here on Monday, no line. its good, but no...",0,D1_nrBr4dOrs7M82OaBRwQ,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,8.0,4.0,False,-1.0,0.5625,8.0
2,iCQpiavjjPzJ5_3gPD5Ebg,0,2016-05-22,0,nsB9JAeghk0zOaSulSm9Yw,5,This place is truly a secret! Its so hidden t...,0,jEjc8HxeCeI22M4di0Y1aw,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,1.0,4.0,False,1.0,1.0000,1.0
3,iCQpiavjjPzJ5_3gPD5Ebg,0,2014-12-30,0,pcn01EMERpCpHEcoaohdEg,5,This SECRET PIZZA was a secret for about 5 st...,0,RTiaaHSN80VRTLSdNK5rfQ,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,1.0,4.0,False,1.0,1.0000,1.0
4,iCQpiavjjPzJ5_3gPD5Ebg,0,2013-03-24,0,0mFtAPTmInbXHqDjX9eiOg,5,this is the BEST place to grab a quick slice o...,0,hBmJs49zYcXdV4wo8F5iiA,"Pizza, Restaurants","{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",Las Vegas,Secret Pizza,5.0,4.0,False,1.0,-0.1000,5.0


In [ ]:
# tried unsuccesfully to create histograms for user_review_count and avg_star_delta
# rest_df.groupby('user_id')['user_review_count'].value_counts().hist(bins = 100)
# rest_df.hist(column = 'user_id')
# rest_df.groupby('user_id')['avg_star_delta'].value_counts().hist()

In [78]:
# reduce dataframe to only include reviews from users with between 5 and 20 reviews
rest_df2 = rest_df[(rest_df.user_review_count > 5) & (rest_df.user_review_count < 20)]
len(rest_df2)/len(rest_df)

0.23838879246527195

In [101]:
# calculates number of reviews per category
hard = len(rest_df2[(rest_df2['avg_star_delta']<-0.5)])
med = len(rest_df2[(rest_df2['avg_star_delta']<= 0.5) & (rest_df2['avg_star_delta']>= -0.5)])
easy = len(rest_df2[(rest_df2['avg_star_delta']>0.5)])
print(hard)
print(med)
print(easy)
print(len(rest_df2))

156092
618461
183466
958019


In [81]:
df = rest_df2.loc[:,['review_id', 'user_id', 'text', 'avg_star_delta', 'user_review_count']]
# check at some point to make sure all review_id's are unique

df['text'] = df['text'].astype(str)

# Group data by user_id and combine review text
grouped = df.groupby(['user_id', 'avg_star_delta'])['text'].sum()

In [82]:
# Fix dataframe format and indexing
df_by_user = pd.DataFrame(grouped)
df_by_user = grouped_df.add_suffix('_aggregated').reset_index()
df_by_user.head()

,user_id,avg_star_delta,text_aggregated
0,--3WaS23LcIXtxyFULJHTA,-0.318182,"Ummm, due to the star ratings on Yelp we made ..."
1,--4rAAfZnEIAKJE80aIiYg,-0.947368,"In my opinion, this restaurant has the best fo..."
2,--CIuK7sUpaNzalLAlHJKA,0.333333,All is right with the world. After going to th...
3,--DxiDMQgN08E5gTM0aj7Q,-1.083333,I went to this sushi place because of its high...
4,--HCoE1ghaAlcaAfshICgw,0.900000,One of the best buffets I have been to for the...


In [83]:
# create easy/hard reviewer labels
df_by_user['reviewer_label'] = pd.cut(df_by_user['avg_star_delta'], bins=[-5,-0.5,.5,5], include_lowest=False, labels=['hard', 'med', 'easy'])
df_by_user.head()

,user_id,avg_star_delta,text_aggregated,reviewer_label
0,--3WaS23LcIXtxyFULJHTA,-0.318182,"Ummm, due to the star ratings on Yelp we made ...",medium
1,--4rAAfZnEIAKJE80aIiYg,-0.947368,"In my opinion, this restaurant has the best fo...",hard
2,--CIuK7sUpaNzalLAlHJKA,0.333333,All is right with the world. After going to th...,medium
3,--DxiDMQgN08E5gTM0aj7Q,-1.083333,I went to this sushi place because of its high...,hard
4,--HCoE1ghaAlcaAfshICgw,0.900000,One of the best buffets I have been to for the...,easy


In [102]:
hard = sum(df_by_user['reviewer_label'] == 'hard')
med = sum(df_by_user['reviewer_label'] == 'med')
easy = sum(df_by_user['reviewer_label'] == 'easy')
print(hard)
print(med)
print(easy)
print(med/len(df_by_user))

20597
0
21937
0.0


In [86]:
train_data, test_data, train_target, test_target = train_test_split(df_by_user['text_aggregated'],
                                                                    df_by_user['reviewer_label'])

In [87]:
str_test_data = []
for text in test_data:
    str_test_data.append(str(text))

In [88]:
str_train_data = []
for text in train_data:
    str_train_data.append(str(text))

In [89]:
vectorizer = TfidfVectorizer(stop_words='english', binary=True, ngram_range=(1,2)) 
train_features = vectorizer.fit_transform(str_train_data)
test_features = vectorizer.transform(str_test_data)

In [90]:
model = LogisticRegression(C=1e9, solver='lbfgs', multi_class='multinomial', random_state=0)
model.fit(train_features, train_target)
predicted = model.predict(test_features)
    
accuracy = accuracy_score(predicted, test_target)

/Users/evan/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [91]:
accuracy

0.7066456508192515

In [92]:
model2 = LinearSVC(fit_intercept=True, random_state=0)
model2.fit(train_features, train_target)
predicted2 = model2.predict(test_features)
    
accuracy2 = accuracy_score(predicted2, test_target)

In [93]:
accuracy2

0.6901873098493457

In [25]:
# this code saves to csv and loads it back in a weird format with messed up review_stars values
# rest_df.to_csv('show_me_the_data/yelp_business_and_reviews', sep='\t', encoding='utf-8')
# df = pd.read_csv('show_me_the_data/yelp_business_and_reviews',sep='\t',index_col=0, encoding='utf-8')
# df.review_stars.unique()

UnicodeEncodeError: 'ascii' codec can't encode character '\xe0' in position 146: ordinal not in range(128)